In [ ]:
import os
from glob import glob
import pandas as pd

In [6]:
DATASETS = os.environ.get('DATASETS', '/User/markjos/datasets')
LIBRIPHRASE = os.path.join(DATASETS, "LibriPhrase")
LIBRISPEECH = os.path.join(DATASETS, "LibriSpeech")

In [8]:
csv_paths = glob(os.path.join(LIBRIPHRASE, 'data', '*.csv'))
csv_paths

['/Users/markjos/datasets/LibriPhrase/data/libriphrase_diffspk_all_2word.csv',
 '/Users/markjos/datasets/LibriPhrase/data/libriphrase_diffspk_all_3word.csv',
 '/Users/markjos/datasets/LibriPhrase/data/libriphrase_diffspk_all_1word.csv',
 '/Users/markjos/datasets/LibriPhrase/data/libriphrase_diffspk_all_4word.csv',
 '/Users/markjos/datasets/LibriPhrase/data/librispeech_clean_train_100h_all_utt.csv']

In [ ]:
df = pd.read_csv(csv_paths[0])